In [1]:
# Basic imports
import pandas as pd
import numpy as np

# Visualization libraries
import seaborn as sns

# Machine Learning libraries
from sklearn.model_selection import train_test_split

# Loading of the dataset via pandas
kc_data = pd.read_csv("data/King_County_House_prices_dataset.csv")

In [2]:
### Data Cleaning

# We will drop this row
kc_data.drop(15856, axis=0, inplace=True)

# We are calculating the "sqft_basement" by substracting sqft_above of sqft_living
kc_data.eval('sqft_basement = sqft_living - sqft_above', inplace=True)



# Summation of the missing values and calculation of the missing values as a percentage
missing_values = pd.DataFrame(kc_data.isnull().sum(),columns=['count'])
missing_values['percentage'] = (missing_values['count']/kc_data.shape[0]*100).round(2)
missing_values.query('count != 0')



# We replace Nan values in "view" with the most frequent expression (0)
kc_data['view'].fillna(0, inplace=True)


# We replace Nan values in "waterfront" with the most frequent expression (0)
kc_data.waterfront.fillna(0, inplace=True)



# We will create an empty list in which we will store values
last_known_change = []

# For each row in our data frame, we look at what is in the column "yr_renovated".
for idx, yr_re in kc_data.yr_renovated.items():
    # if "yr_renovated" is 0 or contains no value, we store the year of construction of the house in our empty listes ab
    if str(yr_re) == 'nan' or yr_re == 0.0:
        last_known_change.append(kc_data.yr_built[idx])
    # if there is a value other than 0 in the column "yr_renovated", we transfer this value into our new list
    else:
        last_known_change.append(int(yr_re))

# We create a new column and take over the values of our previously created list
kc_data['last_known_change'] = last_known_change

# We delete the "yr_renovated" and "yr_built" columns
kc_data.drop("yr_renovated", axis=1, inplace=True)
kc_data.drop("yr_built", axis=1, inplace=True)

In [6]:
### Feature Engineering
### price per sqft
kc_data['sqft_price'] = (kc_data.price/(kc_data.sqft_living + kc_data.sqft_lot)).round(2)

In [8]:
kc_data[kc_data['price']==kc_data['price'].max()]

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,zipcode,lat,long,sqft_living15,sqft_lot15,last_known_change,sqft_price
7245,6762700020,10/13/2014,7700000.0,6,8.0,12050,27600,2.5,0.0,3.0,...,13,8570,3480,98102,47.6298,-122.323,3940,8800,1987,194.2


In [ ]:
### Distance to the center of wealth
kc_data[kc_data['sqft_price']==kc_data['sqft_price'].max()]

kc_data[kc_data['price']==kc_data['price'].max()]

# Absolute difference of latitude between centre and property
kc_data['delta_lat'] = np.absolute(47.62774- kc_data['lat'])
# Absolute difference of longitude between centre and property
kc_data['delta_long'] = np.absolute(-122.24194-kc_data['long'])
# Distance between centre and property
kc_data['center_distance']= ((kc_data['delta_long']* np.cos(np.radians(47.6219)))**2 
                                   + kc_data['delta_lat']**2)**(1/2)*2*np.pi*6378/360

In [9]:
kc_data.query('waterfront == 1')

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,zipcode,lat,long,sqft_living15,sqft_lot15,last_known_change,sqft_price
49,822039084,3/11/2015,1350000.0,3,2.50,2753,65005,1.0,1.0,2.0,...,9,2165,588,98070,47.4041,-122.451,2680,72513,1953,19.92
230,8096000060,4/13/2015,655000.0,2,1.75,1450,15798,2.0,1.0,4.0,...,7,1230,220,98166,47.4497,-122.375,2030,13193,1978,37.98
246,2025069065,9/29/2014,2400000.0,4,2.50,3650,8354,1.0,1.0,4.0,...,9,1830,1820,98074,47.6338,-122.072,3120,18841,2000,199.93
264,2123039032,10/27/2014,369900.0,1,0.75,760,10079,1.0,1.0,4.0,...,5,760,0,98070,47.4683,-122.438,1230,14267,1936,34.13
300,3225069065,6/24/2014,3080000.0,4,5.00,4550,18641,1.0,1.0,4.0,...,10,2600,1950,98074,47.6053,-122.077,4550,19508,2002,132.81
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19968,2025069140,3/17/2015,1900000.0,3,2.50,2830,4334,3.0,1.0,4.0,...,10,2830,0,98074,47.6318,-122.071,2830,38211,2006,265.21
20309,518500480,8/11/2014,3000000.0,3,3.50,4410,10756,2.0,1.0,4.0,...,11,3430,980,98056,47.5283,-122.205,3550,5634,2014,197.81
20751,8043700105,4/17/2015,2300000.0,4,4.00,4360,8175,2.5,1.0,4.0,...,10,3940,420,98008,47.5724,-122.104,2670,8525,2007,183.49
21185,518500460,10/8/2014,2230000.0,3,3.50,3760,5634,2.0,1.0,4.0,...,11,2830,930,98056,47.5285,-122.205,3560,5762,2014,237.39


In [ ]:
# This function helps us to calculate the distance between the house overlooking the seafront and the other houses.
def dist(long, lat, ref_long, ref_lat):
    '''dist computes the distance in km to a reference location. Input: long and lat of 
    the location of interest and ref_long and ref_lat as the long and lat of the reference location'''
    delta_long = long - ref_long
    delta_lat = lat - ref_lat
    delta_long_corr = delta_long * np.cos(np.radians(ref_lat))
    return ((delta_long_corr)**2 +(delta_lat)**2)**(1/2)*2*np.pi*6378/360

# All houses with "waterfront" are added to the list
water_list= kc_data.query('waterfront == 1')
water_list.head()

water_distance = []
# For each row in our data frame we now calculate the distance to the seafront
for idx, lat in kc_data.lat.iteritems():
    ref_list = []
    for x,y in zip(list(water_list.long), list(water_list.lat)):
        ref_list.append(dist(kc_data.long[idx], kc_data.lat[idx],x,y).min())
    water_distance.append(min(ref_list))

# wir erstellen eine neue Spalte und übernehmen die Werte unserer vorher erstellten Liste
kc_data['water_distance'] = water_distance



In [ ]:
### Modelling preparation
# We have to remove these columns because of Data Leakage: price or because they do not provide prognostic information.
drop_lst = ['price', 'sqft_price', 'date', 'delta_lat', 'delta_long',]

# we would like to consider all variables except the ones mentioned above
all_features = [x for x in kc_data.columns if x not in drop_lst]

# X contains all descriptive variables defined above
X = kc_data[all_features]

# we define y (our dependent variable): we take the price
y = kc_data.price

### Move this to the front!
# We separate our data into train and test data. In the process, 30 % of the data is used for the subsequent testing of the prognostic quality.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


